# Optimum Neighborhoods to open an Italian restaurant in Toronto

# 1. Introduction/Business Problem

### 1.1. Description of the problem

The problem consist in identifying the optimum place to open an Italian restaurant in the city of Toronto, Canada. <br> 
The target audience of this project are entrepreneurs of Italian food or owners of italian restaurants. <br>
More specifically, stakeholders who are interested in indentyfing the best Neighborhood candidates in Toronto to open a new Italian restaurant, so with the best group of candidates decide finally where to open it. 

### 1.2. Discussion of the background

One important aspect for a restaurant succes is the place where it is located. Of course, the food and service are very important, however the location can be as crucial specially in the early years. Therefore it is necessary to make a good analysis of the location, in a first step of the neighborhood, of the restaurant before starting the operation. Thats why the stakeholders would care about this project.  <br> 
Some key aspects to take into account for this analysis are:
1. Parking.
2. Visibility.
3. Number of people who can pass near the restaurant.
4. Income of the Neighborhood.
5. Presence of near similiar restaurants.
6. Presence of other business that can attract people to the restaurant: Stadiums, parks, theaters, medical majors.




# 2. Data

### 2.1. Data description

For this analysis the following data is going to be used:
<ol>
<li>List of Neighborhoods in toronto: from Wikipedia</li>
<li>Latitude and longitude coordinates of neighborhoods: from Geospatial data </li> 
<li>Venues near every neighborhood: from Foursquare Api</li>
<li>Population and income of each neighborhood: from Wikipedia demographic data</li>
</ol>

### 2.2. Data usage

The data is going to be used in the following way to solve the problem: 
</ul>
<li>The list of Neighborhoods and its coordinates data are going to be merged to identify the location of each neighborhood.</li>
<li>The population and income data of each neighborhood is going to be also merged with the previous data.</li>
<li>The venues data of every neighborhood is going to be clasified in the following way: Parking and presence of other business that can attract people to the restaurant (like stadiums, theaters, medical majors) are going to be count as type A and similar restuarants (Italian food) are going to be count as type B.  </li>
<li>Finally, K-mean machine learning will be used to clauster the neighborhoods with this data: Population, Income, Type A venues and Type B venues. </li>
</ul>

We propose that is a benefit to the new restaurant to have higher values in population, income, type A venues and lower in type B venues.